# 第七章 型態、方法與介面 - Part 1: 型態與方法

本章將深入探討 Go 語言的型態系統和方法。型態和方法是 Go 物件導向程式設計的基礎，理解這些概念對於編寫優雅、可維護的 Go 程式碼至關重要。

## Go 的型態

Go 允許我們定義自己的型態，這是建構複雜程式的基礎。型態系統提供了型態安全、程式碼組織和抽象的能力。

### 基本型態定義
使用 `type` 關鍵字來定義新型態：

```go
type TypeName UnderlyingType
```

### 自定義型態的優勢
- **型態安全**：防止不同概念的值被混用
- **語義清晰**：型態名稱表達業務意義
- **方法附加**：可以為自定義型態添加方法
- **介面實現**：可以實現介面


### 宣告位置
Go可以讓你在任何一個區塊宣告type, 但只能在那個區塊生效

### 型態分類
Go的type只有分抽象跟具體兩種, 沒有混和型態
- **抽象type** ; interface, 定義應該要做什麼
- **具體** "   : 其他型態, 儲存資料, 以及定義該怎麼做

不像是Java有Abstract class, Go沒有這種東西

In [ ]:
package main

import "fmt"

// 基於內建型態定義新型態
type UserID int
type Email string
type Temperature float64

// 型態別名 (Type Alias) - 與原型態完全相同
type MyInt = int

// 結構體型態
type Person struct {
    ID    UserID // 使用自定義型態
    Name  string
    Email Email
}

func main() {
    // 創建自定義型態的值
    var userID UserID = 123
    var email Email = "user@example.com"
    var temp Temperature = 36.5
    
    fmt.Printf("UserID: %d\n", userID)     // 輸出: UserID: 123
    fmt.Printf("Email: %s\n", email)       // 輸出: Email: user@example.com
    fmt.Printf("Temperature: %.1f°C\n", temp) // 輸出: Temperature: 36.5°C
    
    // 型態別名與原型態可以直接互換
    var a MyInt = 10
    var b int = 20
    a = b // 可以直接賦值，因為 MyInt 是 int 的別名
    fmt.Printf("MyInt: %d\n", a) // 輸出: MyInt: 20
    
    // 自定義型態需要明確轉換
    var id1 UserID = 100
    var id2 int = int(id1) // 需要明確轉換
    fmt.Printf("Converted ID: %d\n", id2) // 輸出: Converted ID: 100
    
    // 創建 Person 結構體
    p := Person{
        ID:    UserID(456),
        Name:  "Alice",
        Email: Email("alice@example.com"),
    }
    fmt.Printf("Person: %+v\n", p) // 輸出: Person: {ID:456 Name:Alice Email:alice@example.com}
}

## 方法

方法是與特定型態關聯的函式。在 Go 中，方法通過接收器 (receiver) 與型態綁定。

### 方法語法
```go
func (接收器 型態) 方法名(參數) 回傳值 {
    // 方法實作
}
```

### 方法 vs 函式
- **方法**：附屬於型態，通過接收器調用
- **函式**：獨立存在，直接調用

### 方法的優勢
- **封裝性**：將相關的行為與資料組合在一起
- **命名空間**：不同型態可以有相同名稱的方法
- **介面實現**：方法可以實現介面

### 宣告位置
Method必須宣告在跟type在一個package裡面, Go允許你在同一個package的不同檔案宣告method, 但慣例上放在一起是比較好的習慣

### Receiver Naming
Receiver 應該要跟著type的第一個字母的小寫來做命名, 不要用this or self
ex: (p Person) or (b Book) 

### method 不能overloadding
跟function一樣, method也不能overloading

In [ ]:
package main

import (
    "fmt"
    "strings"
)

type Person struct {
    Name string
    Age  int
}

type Counter struct {
    value int
}

// Person 的方法
func (p Person) GetInfo() string {
    return fmt.Sprintf("%s (%d years old)", p.Name, p.Age)
}

func (p Person) IsAdult() bool {
    return p.Age >= 18
}

func (p Person) Greet() string {
    return "Hello, I'm " + p.Name
}

// Counter 的方法 (注意：同樣可以有 GetInfo 方法)
func (c Counter) GetInfo() string {
    return fmt.Sprintf("Counter value: %d", c.value)
}

func (c Counter) IsZero() bool {
    return c.value == 0
}

// 獨立的函式 (不是方法)
func FormatName(name string) string {
    return strings.Title(strings.ToLower(name))
}

func main() {
    // 創建 Person 實例
    p := Person{Name: "Alice", Age: 25}
    
    // 調用 Person 的方法
    fmt.Println(p.GetInfo())  // 輸出: Alice (25 years old)
    fmt.Println(p.IsAdult())  // 輸出: true
    fmt.Println(p.Greet())    // 輸出: Hello, I'm Alice
    
    // 創建 Counter 實例
    c := Counter{value: 42}
    
    // 調用 Counter 的方法 (同名方法不衝突)
    fmt.Println(c.GetInfo())  // 輸出: Counter value: 42
    fmt.Println(c.IsZero())   // 輸出: false
    
    // 調用獨立函式
    formatted := FormatName("JOHN DOE")
    fmt.Println(formatted)    // 輸出: John Doe
    
    // 方法可以鏈式調用（如果回傳適當的型態）
    info := Person{Name: "Bob", Age: 16}.GetInfo()
    fmt.Println(info)         // 輸出: Bob (16 years old)
}

## 指標接收子與值接收子

Go 的方法接收器可以是值接收器或指標接收器，選擇哪一種會影響方法的行為和性能。

### 值接收器 (Value Receiver)
- 接收型態的副本
- 不能修改原始值
- 適合小型結構體或不需修改的操作

### 指標接收器 (Pointer Receiver) 
- 接收型態的指標
- 可以修改原始值
- 避免大型結構體的複製開銷

### 選擇指標接收器的情況

#### 一定要使用Pointer method
1. 方法需要**修改**接收器
2. 你的方法需要處理**nil**

#### 建議使用Pointer method
1. 接收器是大型結構體 (> 1MB)
2. 為了一致性（其他方法已使用指標接收器）

In [ ]:
package main

import "fmt"

type BankAccount struct {
    owner   string
    balance float64
}

type Rectangle struct {
    width  float64
    height float64
}

// 值接收器 - 不修改原始值
func (r Rectangle) Area() float64 {
    return r.width * r.height
}

func (r Rectangle) Perimeter() float64 {
    return 2 * (r.width + r.height)
}

// 值接收器嘗試修改（無效）
func (r Rectangle) TryScale(factor float64) {
    r.width *= factor   // 只修改副本，不影響原始值
    r.height *= factor
}

// 指標接收器 - 可以修改原始值
func (ba *BankAccount) Deposit(amount float64) {
    if amount > 0 {
        ba.balance += amount // 修改原始值
    }
}

func (ba *BankAccount) Withdraw(amount float64) bool {
    if amount > 0 && ba.balance >= amount {
        ba.balance -= amount // 修改原始值
        return true
    }
    return false
}

// 指標接收器 - 查詢方法
func (ba *BankAccount) GetBalance() float64 {
    return ba.balance
}

// 值接收器 - 查詢方法
func (ba BankAccount) GetOwner() string {
    return ba.owner
}

func main() {
    // 測試值接收器
    rect := Rectangle{width: 5, height: 3}
    fmt.Printf("Original rectangle: %+v\n", rect) // 輸出: Original rectangle: {width:5 height:3}
    
    area := rect.Area()
    perimeter := rect.Perimeter()
    fmt.Printf("Area: %.1f, Perimeter: %.1f\n", area, perimeter) // 輸出: Area: 15.0, Perimeter: 16.0
    
    // 嘗試修改（無效）
    rect.TryScale(2)
    fmt.Printf("After TryScale: %+v\n", rect) // 輸出: After TryScale: {width:5 height:3} (沒有改變)
    
    // 測試指標接收器
    account := BankAccount{owner: "Alice", balance: 1000}
    fmt.Printf("Initial account: %+v\n", account) // 輸出: Initial account: {owner:Alice balance:1000}
    
    // 存款
    account.Deposit(500)
    fmt.Printf("After deposit: Balance = %.2f\n", account.GetBalance()) // 輸出: After deposit: Balance = 1500.00
    
    // 提款
    success := account.Withdraw(200)
    fmt.Printf("Withdraw success: %t, Balance = %.2f\n", success, account.GetBalance()) // 輸出: Withdraw success: true, Balance = 1300.00
    
    // 提款失敗
    success = account.Withdraw(2000)
    fmt.Printf("Withdraw success: %t, Balance = %.2f\n", success, account.GetBalance()) // 輸出: Withdraw success: false, Balance = 1300.00
    
    // Go 的語法糖：自動取址和解址
    fmt.Printf("Owner: %s\n", account.GetOwner()) // 值接收器，但可以直接調用
    
    // 指標也可以調用值接收器的方法
    accountPtr := &account
    fmt.Printf("Owner via pointer: %s\n", accountPtr.GetOwner()) // 自動解址
}

## 語法糖 (syntactic sugar)

### 呼叫method, Go會自動幫你轉換成 receiver 的type


我們宣告一個struct, 分別有一個 value receiver, 一個 pointer receiver

```go
type Counter struct {
    total int
    lastUpdated time.Time
}

func (c *Counter) Increment() {
    c.total += 1
    c.lastUpdated = time.Now
}

func (c Counter) String string() {
    return fmt.Sprintf("total: %d, last updated: %v, c.total, c.lastUpdated")
}


var c Count // all zero instance
fmt.Println(c.string()) // total = 0
c.Increment //呼叫 pointer type => 不會出錯, 會自動幫你轉 (&c).Increment
fmt.Println(c.string()) // total = 1

```
### 當你呼叫method時, 如果你的變數是value type, 但method是pointer receiver, Go會幫你自動轉換
**c.Increment -> (&c).Increment**

### 反過來說, 如果你的變數是pointer type, 但method是value receiver, Go也會幫你轉換. 


### 語法糖陷阱 : 如果你的value type function有呼叫到pointer receiver method, 那麼你是對著副本在呼叫的

```go
func doUpdateWrong(c Counter){ // **注意這邊傳送的是 C 的 Copy, 所以改變的也是copy version **
    c.Increment()
    fmt.Println(s.string()) // total = 1 
}

func doUpdateRight(C *Counter) {
    c.Increment()
    fmt.Println(s.string()) // total = 1
}

func main(){
    var c Counter //total = 0
    doUpdateWrong(c) 
    fmt.Println("in main:",c.string()) // 0
    doUpdateRight(&c)
    fmt.Println("in main:", c.string() ) // 1
}

```
### 不要寫 Getter & Setter, 直接存取欄位就好


## 為nil實例編寫方法

對著nil instance呼叫方法, 如果呼叫的是

**Value Receiver** : 得到Panic

**Pointer Receiver** : 可以用if先判斷是不是nil, 再做後續處理

## 為了程式碼重複的方法

方法可以幫助我們避免程式碼重複，通過將共同的行為封裝到型態的方法中，我們可以提高程式碼的可重用性和維護性。

### 方法重用的策略
- **共同行為抽象**：將重複出現的邏輯抽象成方法
- **輔助方法**：創建私有方法來支援公開方法
- **方法鏈**：設計可以連續調用的方法
- **工廠方法**：提供統一的對象創建介面

In [ ]:
package main

import (
    "fmt"
    "strings"
    "time"
)

type Logger struct {
    prefix string
    buffer []string
}

type User struct {
    ID        int
    FirstName string
    LastName  string
    Email     string
    CreatedAt time.Time
}

// Logger 的工廠方法
func NewLogger(prefix string) *Logger {
    return &Logger{
        prefix: prefix,
        buffer: make([]string, 0),
    }
}

// 私有輔助方法：格式化時間戳
func (l *Logger) formatTimestamp() string {
    return time.Now().Format("2006-01-02 15:04:05")
}

// 私有輔助方法：創建完整的日誌條目
func (l *Logger) createEntry(level, message string) string {
    timestamp := l.formatTimestamp()
    return fmt.Sprintf("[%s] %s [%s] %s", timestamp, l.prefix, level, message)
}

// 公開方法：使用共同的輔助方法
func (l *Logger) Info(message string) {
    entry := l.createEntry("INFO", message)
    l.buffer = append(l.buffer, entry)
    fmt.Println(entry)
}

func (l *Logger) Warning(message string) {
    entry := l.createEntry("WARN", message)
    l.buffer = append(l.buffer, entry)
    fmt.Println(entry)
}

func (l *Logger) Error(message string) {
    entry := l.createEntry("ERROR", message)
    l.buffer = append(l.buffer, entry)
    fmt.Println(entry)
}

// 方法鏈：返回自身指標以支援連續調用
func (l *Logger) SetPrefix(prefix string) *Logger {
    l.prefix = prefix
    return l
}

func (l *Logger) Clear() *Logger {
    l.buffer = l.buffer[:0]
    return l
}

func (l *Logger) GetLogs() []string {
    return l.buffer
}

// User 的工廠方法
func NewUser(firstName, lastName, email string) *User {
    return &User{
        FirstName: firstName,
        LastName:  lastName,
        Email:     email,
        CreatedAt: time.Now(),
    }
}

// 私有輔助方法：標準化名稱格式
func (u *User) formatName(name string) string {
    return strings.Title(strings.ToLower(strings.TrimSpace(name)))
}

// 使用輔助方法的公開方法
func (u *User) SetFirstName(firstName string) {
    u.FirstName = u.formatName(firstName)
}

func (u *User) SetLastName(lastName string) {
    u.LastName = u.formatName(lastName)
}

func (u *User) FullName() string {
    return u.FirstName + " " + u.LastName
}

func (u *User) DisplayInfo() string {
    return fmt.Sprintf("User #%d: %s <%s> (Created: %s)", 
        u.ID, u.FullName(), u.Email, u.CreatedAt.Format("2006-01-02"))
}

func main() {
    // 使用工廠方法創建 Logger
    logger := NewLogger("APP")
    
    // 所有方法都使用相同的格式化邏輯
    logger.Info("Application started")
    logger.Warning("This is a warning")
    logger.Error("An error occurred")
    
    // 方法鏈示例
    logger.SetPrefix("WEB").Info("Web server started")
    
    fmt.Println("\n--- Log Buffer ---")
    for i, log := range logger.GetLogs() {
        fmt.Printf("%d: %s\n", i+1, log)
    }
    
    // User 示例
    fmt.Println("\n--- User Example ---")
    user := NewUser("  JOHN  ", "  doe  ", "john.doe@example.com")
    user.ID = 1
    
    fmt.Println("Before formatting:", user.FirstName, user.LastName) // 輸出: Before formatting:   JOHN     doe  
    
    // 使用標準化方法
    user.SetFirstName("  JOHN  ")
    user.SetLastName("  doe  ")
    
    fmt.Println("After formatting:", user.FirstName, user.LastName) // 輸出: After formatting: John Doe
    fmt.Println("Full name:", user.FullName())                      // 輸出: Full name: John Doe
    fmt.Println(user.DisplayInfo())                                 // 輸出: User #1: John Doe <john.doe@example.com> (Created: 2024-XX-XX)
}

## 方法也是函式

在 Go 中，方法本質上是一種特殊的函式，它們可以像函式一樣被引用、傳遞和調用。

```go
type Adder struct {
    start int
}

func (a Adder) AddTo(val int) {
    return a.start + val
}

```

### 方法值 (Method Values)
當你引用一個方法時，Go 會創建一個「方法值」，這是一個已經綁定了接收器(instance)的函式。

```go
type Adder struct {
    start int
}

func (a *Adder) AddTo(val int) {
    return a.start + val
}

func (a Adder) String(val int) {
    return "Adder{start=",a.start,"}"
}

//使用
base = Adder(10)

addBase = (&base).AddTo

fmt.println("addBase=",addBase(5))

showBase = base.String()

fmt.println("showBase=", showBase())

```

### 方法表達式 (Method Expressions)
方法表達式是一個未綁定接收器的函式，調用時需要明確傳入接收器(instance)作為第一個參數。

```go
type Adder struct {
    start int
}

func (a *Adder) AddTo(val int) {
    return a.start + val
}

func (a Adder) String(val int) {
    return "Adder{start=",a.start,"}"
}

base = Adder{start: 10}

add = Adder.AddTo

fmt.Println("add=", add(base, 5))

show = Adder.String

fmt.Println("show=", show(base))


```


In [3]:
package main

import "fmt"

type Calculator struct {
    value float64
}

func (c *Calculator) Add(n float64) {
    c.value += n
}

func (c *Calculator) Multiply(n float64) {
    c.value *= n
}

func (c Calculator) GetValue() float64 {
    return c.value
}

func (c Calculator) String() string {
    return fmt.Sprintf("Calculator{value: %.2f}", c.value)
}

// 接受函式作為參數的函式
func applyOperation(calc *Calculator, operation func(float64), operand float64) {
    fmt.Printf("Before operation: %s\n", calc)
    operation(operand)
    fmt.Printf("After operation: %s\n", calc)
}

// 接受方法表達式的函式
func applyWithExpression(calc *Calculator, expr func(*Calculator, float64), operand float64) {
    fmt.Printf("Using expression - Before: %s\n", calc)
    expr(calc, operand)
    fmt.Printf("Using expression - After: %s\n", calc)
}

func main() {
    calc := &Calculator{value: 10}
    
    fmt.Println("=== 基本方法調用 ===")
    fmt.Printf("Initial: %s\n", calc)  // 輸出: Initial: Calculator{value: 10.00}
    
    calc.Add(5)
    fmt.Printf("After Add(5): %s\n", calc)  // 輸出: After Add(5): Calculator{value: 15.00}
    
    calc.Multiply(2)
    fmt.Printf("After Multiply(2): %s\n", calc)  // 輸出: After Multiply(2): Calculator{value: 30.00}
    
    fmt.Println("\n=== 方法值 (Method Values) ===")
    // 方法值：綁定了特定接收器的函式
    addMethod := calc.Add        // 方法值，綁定了 calc
    multiplyMethod := calc.Multiply
    
    // 方法值的型態
    fmt.Printf("addMethod type: %T\n", addMethod)  // 輸出: addMethod type: func(float64)
    
    // 直接調用方法值
    addMethod(10)  // 等同於 calc.Add(10)
    fmt.Printf("After addMethod(10): %s\n", calc)  // 輸出: After addMethod(10): Calculator{value: 40.00}
    
    // 將方法值傳遞給其他函式
    applyOperation(calc, addMethod, 5)    // 會調用 calc.Add(5)
    applyOperation(calc, multiplyMethod, 1.5)  // 會調用 calc.Multiply(1.5)
    
    fmt.Println("\n=== 方法表達式 (Method Expressions) ===")
    // 方法表達式：未綁定接收器的函式
    addExpr := (*Calculator).Add        // 方法表達式
    //multiplyExpr := (*Calculator).Multiply
    
    fmt.Printf("addExpr type: %T\n", addExpr)  // 輸出: addExpr type: func(*main.Calculator, float64)
    
    // 調用方法表達式需要明確傳入接收器
    addExpr(calc, 3)  // 第一個參數是接收器
    fmt.Printf("After addExpr(calc, 3): %s\n", calc)  // 輸出: After addExpr(calc, 3): Calculator{value: 70.50}
    
    // 將方法表達式傳遞給函式
    applyWithExpression(calc, addExpr, 4.5)
    
    fmt.Println("\n=== 值接收器的方法表達式 ===")
    getValueExpr := Calculator.GetValue  // 值接收器的方法表達式
    fmt.Printf("getValueExpr type: %T\n", getValueExpr)  // 輸出: getValueExpr type: func(main.Calculator) float64
    
    // 調用值接收器的方法表達式
    value := getValueExpr(*calc)  // 需要解引用指標
    fmt.Printf("Current value: %.2f\n", value)  // 輸出: Current value: 75.00
    
    fmt.Println("\n=== 方法作為一等公民 ===")
    // 將多個方法存儲在切片中
    operations := []func(float64){
        calc.Add,
        calc.Multiply,
    }
    
    operands := []float64{2, 0.5}
    operationNames := []string{"Add", "Multiply"}
    
    for i, op := range operations {
        fmt.Printf("Applying %s with %.1f\n", operationNames[i], operands[i])
        op(operands[i])
        fmt.Printf("Result: %s\n", calc)
    }
    // 輸出: Applying Add with 2.0
    //      Result: Calculator{value: 77.00}
    //      Applying Multiply with 0.5
    //      Result: Calculator{value: 38.50}
}

=== 基本方法調用 ===
Initial: Calculator{value: 10.00}
After Add(5): Calculator{value: 15.00}
After Multiply(2): Calculator{value: 30.00}

=== 方法值 (Method Values) ===
addMethod type: func(float64)
After addMethod(10): Calculator{value: 40.00}
Before operation: Calculator{value: 40.00}
After operation: Calculator{value: 45.00}
Before operation: Calculator{value: 45.00}
After operation: Calculator{value: 67.50}

=== 方法表達式 (Method Expressions) ===
addExpr type: func(*main.Calculator, float64)
After addExpr(calc, 3): Calculator{value: 70.50}
Using expression - Before: Calculator{value: 70.50}
Using expression - After: Calculator{value: 75.00}

=== 值接收器的方法表達式 ===
getValueExpr type: func(main.Calculator) float64
Current value: 75.00

=== 方法作為一等公民 ===
Applying Add with 2.0
Result: Calculator{value: 77.00}
Applying Multiply with 0.5
Result: Calculator{value: 38.50}


## 函式 vs. 方法

雖然方法和函式在 Go 中都用來組織程式碼邏輯，但它們有著重要的區別。

### 主要差異
- **函式**：獨立存在，不屬於任何型態
- **方法**：屬於特定的型態，有接收器

### 使用場景
- **函式**：處理純邏輯、工具功能、不需要狀態的操作
- **方法**：操作特定型態的資料、實現介面、封裝行為

In [ ]:
package main

import (
    "fmt"
    "math"
    "strings"
    "time"
)

// 型態定義
type Circle struct {
    radius float64
}

type Person struct {
    name string
    age  int
}

// =============================================================================
// 函式：獨立的功能，不依賴特定型態
// =============================================================================

// 純數學計算函式
func CalculateCircleArea(radius float64) float64 {
    return math.Pi * radius * radius
}

func CalculateCirclePerimeter(radius float64) float64 {
    return 2 * math.Pi * radius
}

// 字串處理工具函式
func FormatName(firstName, lastName string) string {
    return strings.Title(firstName) + " " + strings.Title(lastName)
}

func ValidateEmail(email string) bool {
    return strings.Contains(email, "@") && strings.Contains(email, ".")
}

// 時間處理函式
func FormatDuration(d time.Duration) string {
    hours := int(d.Hours())
    minutes := int(d.Minutes()) % 60
    return fmt.Sprintf("%dh %dm", hours, minutes)
}

// =============================================================================
// 方法：綁定到特定型態，操作該型態的資料
// =============================================================================

// Circle 的方法
func (c Circle) Area() float64 {
    return math.Pi * c.radius * c.radius
}

func (c Circle) Perimeter() float64 {
    return 2 * math.Pi * c.radius
}

func (c *Circle) Scale(factor float64) {
    c.radius *= factor
}

func (c Circle) String() string {
    return fmt.Sprintf("Circle{radius: %.2f}", c.radius)
}

// Person 的方法
func (p Person) GetAge() int {
    return p.age
}

func (p *Person) SetName(name string) {
    p.name = strings.Title(name)
}

func (p *Person) HaveBirthday() {
    p.age++
}

func (p Person) IsAdult() bool {
    return p.age >= 18
}

func (p Person) Introduce() string {
    return fmt.Sprintf("Hi, I'm %s and I'm %d years old", p.name, p.age)
}

func main() {
    fmt.Println("=== 函式示例 ===")
    
    // 使用函式：傳入參數，獲得結果
    radius := 5.0
    area := CalculateCircleArea(radius)
    perimeter := CalculateCirclePerimeter(radius)
    
    fmt.Printf("Circle with radius %.1f:\n", radius)
    fmt.Printf("Area (function): %.2f\n", area)         // 輸出: Area (function): 78.54
    fmt.Printf("Perimeter (function): %.2f\n", perimeter) // 輸出: Perimeter (function): 31.42
    
    // 字串處理函式
    formatted := FormatName("john", "doe")
    isValid := ValidateEmail("user@example.com")
    fmt.Printf("Formatted name: %s\n", formatted) // 輸出: Formatted name: John Doe
    fmt.Printf("Email valid: %t\n", isValid)      // 輸出: Email valid: true
    
    // 時間處理函式
    duration := 2*time.Hour + 30*time.Minute
    formatted_duration := FormatDuration(duration)
    fmt.Printf("Duration: %s\n", formatted_duration) // 輸出: Duration: 2h 30m
    
    fmt.Println("\n=== 方法示例 ===")
    
    // 使用方法：通過接收器調用
    circle := Circle{radius: 5.0}
    fmt.Printf("Circle: %s\n", circle)                    // 輸出: Circle: Circle{radius: 5.00}
    fmt.Printf("Area (method): %.2f\n", circle.Area())     // 輸出: Area (method): 78.54
    fmt.Printf("Perimeter (method): %.2f\n", circle.Perimeter()) // 輸出: Perimeter (method): 31.42
    
    // 修改接收器的方法
    circle.Scale(2)
    fmt.Printf("After scaling: %s\n", circle)             // 輸出: After scaling: Circle{radius: 10.00}
    
    // Person 方法示例
    person := Person{name: "alice", age: 17}
    fmt.Printf("Initial: %s\n", person.Introduce())      // 輸出: Initial: Hi, I'm alice and I'm 17 years old
    fmt.Printf("Is adult: %t\n", person.IsAdult())       // 輸出: Is adult: false
    
    person.SetName("alice smith")
    person.HaveBirthday()
    fmt.Printf("After updates: %s\n", person.Introduce()) // 輸出: After updates: Hi, I'm Alice Smith and I'm 18 years old
    fmt.Printf("Is adult: %t\n", person.IsAdult())        // 輸出: Is adult: true
    
    fmt.Println("\n=== 對比總結 ===")
    
    fmt.Println("函式特點：")
    fmt.Println("- 獨立存在，不依賴特定型態")
    fmt.Println("- 通過參數接收資料")
    fmt.Println("- 通常處理純邏輯或工具性功能")
    fmt.Println("- 可以在套件級別定義")
    
    fmt.Println("\n方法特點：")
    fmt.Println("- 綁定到特定型態")
    fmt.Println("- 通過接收器存取資料")
    fmt.Println("- 封裝與該型態相關的行為")
    fmt.Println("- 可以實現介面")
    fmt.Println("- 提供物件導向的程式設計方式")
}

## 型態宣告不是繼承

Go 的型態系統與傳統物件導向語言不同，它不支援繼承，而是採用組合的方式。

### 重要概念
- Go 的型態宣告創建的是**全新的型態**，不是繼承
- 新型態與底層型態在型態檢查上是**不同的**
- 方法不會自動「繼承」到新型態
- 需要明確的型態轉換

### 與繼承的區別
- **繼承**：子類是父類的一種特殊形式
- **Go 型態**：完全獨立的型態，只是共享底層表示

### 宣告type, 有沒有等號有差別
- **type 型態 = 變數**：只是Alias, 變數跟原本的type一樣, 可以互相assign
- **type 型態 變數**：變數跟原本的type不同, 可以用來賦予新的method
```go
// 基於 Score 定義新型態（不是繼承）
type ExamScore Score     // 新型態，不繼承 Score 的方法


// 型態別名（完全等同於原型態）
type ScoreAlias = Score // 新型態，可以使用 Score 的方法
```

In [4]:
package main

import "fmt"

// 基礎型態
type Score int

// Score 的方法
func (s Score) IsPass() bool {
    return s >= 60
}

func (s Score) Grade() string {
    switch {
    case s >= 90:
        return "A"
    case s >= 80:
        return "B"
    case s >= 70:
        return "C"
    case s >= 60:
        return "D"
    default:
        return "F"
    }
}

// 基於 Score 定義新型態（不是繼承）
type ExamScore Score     // 新型態，不繼承 Score 的方法
type QuizScore Score     // 另一個新型態

// 必須為 ExamScore 重新定義方法
func (es ExamScore) IsPass() bool {
    return es >= 70  // 考試及格分數更高
}

func (es ExamScore) Grade() string {
    switch {
    case es >= 95:
        return "A+"
    case es >= 90:
        return "A"
    case es >= 80:
        return "B"
    case es >= 70:
        return "C"
    default:
        return "F"
    }
}

// QuizScore 有不同的方法實現
func (qs QuizScore) IsPass() bool {
    return qs >= 50  // 小考及格分數較低
}

func (qs QuizScore) Difficulty() string {
    if qs >= 80 {
        return "Hard"
    } else if qs >= 60 {
        return "Medium"
    }
    return "Easy"
}

// 型態別名（完全等同於原型態）
type ScoreAlias = Score

// 函式參數型態檢查
func PrintScoreInfo(s Score) {
    fmt.Printf("Score: %d, Pass: %t, Grade: %s\n", s, s.IsPass(), s.Grade())
}

func PrintExamInfo(es ExamScore) {
    fmt.Printf("Exam Score: %d, Pass: %t, Grade: %s\n", es, es.IsPass(), es.Grade())
}

func main() {
    fmt.Println("=== 型態獨立性示例 ===")
    
    // 創建不同型態的值
    var score Score = 75
    var examScore ExamScore = 75
    var quizScore QuizScore = 75
    var scoreAlias ScoreAlias = 75
    
    // 相同的數值，不同的行為
    fmt.Printf("Score (75): Pass = %t, Grade = %s\n", score.IsPass(), score.Grade())
    // 輸出: Score (75): Pass = true, Grade = C
    
    fmt.Printf("ExamScore (75): Pass = %t, Grade = %s\n", examScore.IsPass(), examScore.Grade())
    // 輸出: ExamScore (75): Pass = true, Grade = C
    
    fmt.Printf("QuizScore (75): Pass = %t, Difficulty = %s\n", quizScore.IsPass(), quizScore.Difficulty())
    // 輸出: QuizScore (75): Pass = true, Difficulty = Medium
    
    fmt.Printf("ScoreAlias (75): Pass = %t, Grade = %s\n", scoreAlias.IsPass(), scoreAlias.Grade())
    // 輸出: ScoreAlias (75): Pass = true, Grade = C
    
    fmt.Println("\n=== 型態轉換需求 ===")
    
    // 不同型態之間不能直接賦值
    // score = examScore  // 編譯錯誤！
    
    // 需要明確轉換
    score = Score(examScore)  // 明確轉換
    fmt.Printf("Converted score: %d\n", score) // 輸出: Converted score: 75
    
    // 型態別名可以直接賦值
    score = scoreAlias  // OK，因為 ScoreAlias = Score
    scoreAlias = score  // OK
    
    fmt.Println("\n=== 函式參數型態檢查 ===")
    
    PrintScoreInfo(score)     // OK
    PrintScoreInfo(scoreAlias) // OK，型態別名
    // PrintScoreInfo(examScore)  // 編譯錯誤！不同型態
    
    PrintScoreInfo(Score(examScore))  // OK，明確轉換
    PrintExamInfo(examScore)          // OK
    PrintExamInfo(ExamScore(score))   // OK，明確轉換
    
    fmt.Println("\n=== 方法不會繼承 ===")
    
    // ExamScore 沒有自動獲得 Score 的方法
    // 如果 ExamScore 沒有定義 IsPass 方法，會編譯錯誤
    
    fmt.Println("每個型態必須定義自己的方法")
    fmt.Println("方法實現可以不同，體現不同的業務邏輯")
    
    fmt.Println("\n=== 底層型態相同但語義不同 ===")
    
    // 都是基於 int，但代表不同的概念
    type UserID int
    type ProductID int
    type OrderID int
    
    var userID UserID = 123
    var productID ProductID = 123
    var orderID OrderID = 123
    
    // 雖然底層值相同，但型態不同，防止誤用
    // userID = productID    // 編譯錯誤！
    // productID = orderID   // 編譯錯誤！
    
    fmt.Printf("UserID: %d\n", userID)     // 輸出: UserID: 123
    fmt.Printf("ProductID: %d\n", productID) // 輸出: ProductID: 123
    fmt.Printf("OrderID: %d\n", orderID)   // 輸出: OrderID: 123
    
    // 如果需要比較，必須轉換到相同型態
    if int(userID) == int(productID) {
        fmt.Println("底層值相同，但型態不同") // 會執行這行
    }
}

=== 型態獨立性示例 ===
Score (75): Pass = true, Grade = C
ExamScore (75): Pass = true, Grade = C
QuizScore (75): Pass = true, Difficulty = Medium
ScoreAlias (75): Pass = true, Grade = C

=== 型態轉換需求 ===
Converted score: 75

=== 函式參數型態檢查 ===
Score: 75, Pass: true, Grade: C
Score: 75, Pass: true, Grade: C
Score: 75, Pass: true, Grade: C
Exam Score: 75, Pass: true, Grade: C
Exam Score: 75, Pass: true, Grade: C

=== 方法不會繼承 ===
每個型態必須定義自己的方法
方法實現可以不同，體現不同的業務邏輯

=== 底層型態相同但語義不同 ===
UserID: 123
ProductID: 123
OrderID: 123
底層值相同，但型態不同


## 型態是可執行的說明文件

Go 鼓勵使用有意義的型態名稱和結構來表達程式碼的意圖。型態本身就是最好的文檔。

### 自描述型態的重要性
- **型態名稱**應該清楚表達其用途和含義
- **結構體欄位**名稱應該具有描述性
- **方法名稱**應該表達其行為和作用
- **業務規則**可以通過型態系統體現

### 良好型態設計的原則
- 使用領域語言命名型態
- 型態應該表達業務概念
- 避免過於通用的名稱
- 通過型態防止錯誤使用

In [ ]:
package main

import (
    "fmt"
    "time"
)

// ======================================================================
// 不好的設計：型態名稱沒有表達明確意義
// ======================================================================

// type Data struct {  // 太泛用，不知道是什麼資料
//     Value int
//     Info  string
// }

// type ID int  // 太籠統，不知道是什麼的 ID

// ======================================================================
// 好的設計：型態名稱表達明確的業務概念
// ======================================================================

// 金融系統中的型態
type AccountNumber string   // 帳戶號碼
type Amount int64          // 金額（以分為單位）
type Currency string       // 貨幣
type TransactionID string  // 交易 ID

// 用戶系統中的型態
type UserID int64         // 用戶 ID
type Email string         // 電子郵件
type PhoneNumber string   // 電話號碼
type PasswordHash string  // 密碼雜湊

// 訂單系統中的型態
type ProductID string     // 產品 ID
type OrderID string       // 訂單 ID
type Quantity int         // 數量
type Weight float64       // 重量（公克）

// 業務實體結構體
type BankAccount struct {
    Number      AccountNumber  // 通過型態名稱就知道這是帳戶號碼
    Owner       UserID        // 明確指出這是用戶 ID
    Balance     Amount        // 餘額，型態名稱說明這是金額
    Currency    Currency      // 貨幣
    CreatedAt   time.Time     // 創建時間
    IsActive    bool          // 是否活躍
}

type User struct {
    ID              UserID        // 用戶唯一標識
    Email           Email         // 登入郵箱
    Phone           PhoneNumber   // 聯絡電話
    PasswordHash    PasswordHash  // 密碼雜湊值
    RegistrationDate time.Time    // 註冊日期
    LastLoginAt     *time.Time    // 最後登入時間（可為空）
}

type Product struct {
    ID          ProductID  // 產品識別碼
    Name        string     // 產品名稱
    Price       Amount     // 價格
    Currency    Currency   // 計價貨幣
    Weight      Weight     // 商品重量
    InStock     Quantity   // 庫存數量
    IsAvailable bool       // 是否可購買
}

type Order struct {
    ID         OrderID       // 訂單編號
    Customer   UserID        // 客戶 ID
    Items      []OrderItem   // 訂單項目
    TotalAmount Amount       // 總金額
    Status     OrderStatus   // 訂單狀態
    CreatedAt  time.Time     // 下單時間
}

type OrderItem struct {
    Product  ProductID  // 產品 ID
    Quantity Quantity   // 購買數量
    UnitPrice Amount    // 單價
}

// 枚舉型態
type OrderStatus string

const (
    OrderStatusPending   OrderStatus = "pending"    // 待處理
    OrderStatusConfirmed OrderStatus = "confirmed"  // 已確認
    OrderStatusShipped   OrderStatus = "shipped"    // 已出貨
    OrderStatusDelivered OrderStatus = "delivered"  // 已送達
    OrderStatusCancelled OrderStatus = "cancelled"  // 已取消
)

// ======================================================================
// 方法名稱也是文檔的一部分
// ======================================================================

// Amount 的方法：方法名稱表達了操作的意圖
func (a Amount) ToYuan() float64 {  // 轉換為元（從分）
    return float64(a) / 100.0
}

func (a Amount) IsPositive() bool {  // 判斷是否為正數
    return a > 0
}

func (a Amount) Add(other Amount) Amount {  // 金額相加
    return a + other
}

// BankAccount 的方法：業務操作
func (ba *BankAccount) CanWithdraw(amount Amount) bool {  // 判斷能否提款
    return ba.IsActive && ba.Balance >= amount
}

func (ba *BankAccount) Deposit(amount Amount) error {  // 存款操作
    if !ba.IsActive {
        return fmt.Errorf("account is not active")
    }
    if !amount.IsPositive() {
        return fmt.Errorf("deposit amount must be positive")
    }
    ba.Balance = ba.Balance.Add(amount)
    return nil
}

func (ba *BankAccount) GetBalanceInYuan() float64 {  // 獲取餘額（元）
    return ba.Balance.ToYuan()
}

// OrderStatus 的方法
func (os OrderStatus) IsActive() bool {  // 判斷訂單是否活躍
    return os != OrderStatusCancelled && os != OrderStatusDelivered
}

func (os OrderStatus) CanBeCancelled() bool {  // 判斷能否取消
    return os == OrderStatusPending || os == OrderStatusConfirmed
}

// Order 的方法
func (o *Order) CalculateTotal() Amount {  // 計算總金額
    var total Amount
    for _, item := range o.Items {
        total = total.Add(Amount(item.Quantity) * item.UnitPrice)
    }
    return total
}

func (o *Order) CanBeModified() bool {  // 判斷訂單能否修改
    return o.Status.IsActive() && o.Status != OrderStatusShipped
}

func main() {
    fmt.Println("=== 型態作為文檔示例 ===")
    
    // 創建有意義的資料
    user := User{
        ID:              UserID(12345),
        Email:           Email("john@example.com"),
        Phone:           PhoneNumber("+886-912-345-678"),
        PasswordHash:    PasswordHash("$2a$10$..."),
        RegistrationDate: time.Now(),
    }
    
    account := BankAccount{
        Number:    AccountNumber("ACC-001-123456"),
        Owner:     user.ID,  // 型態確保了關聯的正確性
        Balance:   Amount(150000),  // 1500.00 元
        Currency:  Currency("TWD"),
        CreatedAt: time.Now(),
        IsActive:  true,
    }
    
    product := Product{
        ID:          ProductID("PROD-001"),
        Name:        "Go 程式設計書籍",
        Price:       Amount(89900),  // 899.00 元
        Currency:    Currency("TWD"),
        Weight:      Weight(500),  // 500 公克
        InStock:     Quantity(100),
        IsAvailable: true,
    }
    
    // 型態名稱使程式碼自我說明
    fmt.Printf("用戶 ID: %d\n", user.ID)  // 清楚知道這是用戶 ID
    fmt.Printf("帳戶號碼: %s\n", account.Number)  // 清楚知道這是帳戶號碼
    fmt.Printf("餘額: %.2f 元\n", account.GetBalanceInYuan())  // 輸出: 餘額: 1500.00 元
    fmt.Printf("產品價格: %.2f 元\n", product.Price.ToYuan())  // 輸出: 產品價格: 899.00 元
    
    // 業務邏輯通過方法名稱表達
    depositAmount := Amount(50000)  // 500.00 元
    if account.CanWithdraw(depositAmount) {
        fmt.Println("可以提款")  // 方法名稱清楚表達了檢查意圖
    }
    
    err := account.Deposit(depositAmount)
    if err != nil {
        fmt.Printf("存款失敗: %v\n", err)
    } else {
        fmt.Printf("存款成功，新餘額: %.2f 元\n", account.GetBalanceInYuan())
        // 輸出: 存款成功，新餘額: 2000.00 元
    }
    
    // 訂單示例
    order := Order{
        ID:       OrderID("ORD-2024-001"),
        Customer: user.ID,
        Items: []OrderItem{
            {
                Product:   product.ID,
                Quantity:  Quantity(2),
                UnitPrice: product.Price,
            },
        },
        Status:    OrderStatusPending,
        CreatedAt: time.Now(),
    }
    
    order.TotalAmount = order.CalculateTotal()
    
    fmt.Printf("\n訂單 %s:\n", order.ID)
    fmt.Printf("總金額: %.2f 元\n", order.TotalAmount.ToYuan())  // 輸出: 總金額: 1798.00 元
    fmt.Printf("狀態: %s\n", order.Status)  // 輸出: 狀態: pending
    fmt.Printf("可以取消: %t\n", order.Status.CanBeCancelled())  // 輸出: 可以取消: true
    fmt.Printf("可以修改: %t\n", order.CanBeModified())  // 輸出: 可以修改: true
    
    fmt.Println("\n=== 型態設計的優勢 ===")
    fmt.Println("1. 程式碼自我說明：型態名稱就是文檔")
    fmt.Println("2. 防止錯誤：不同概念的 ID 不會混用")
    fmt.Println("3. 業務邏輯清晰：方法名稱表達業務意圖")
    fmt.Println("4. 型態安全：編譯時檢查防止錯誤")
    fmt.Println("5. 維護性好：修改型態定義自動更新相關程式碼")
}

## iota

`iota` 是 Go 提供的常數生成器，在 `const` 宣告中使用，每次出現時自動遞增。它特別適用於定義枚舉類型的常數。

### iota 的基本規則
- 在每個 `const` 區塊中，第一個 `iota` 的值為 0
- 後續每行中的 `iota` 都會自動加 1
- 可以參與運算表達式
- 常用來定義枚舉類型

### 使用模式
- 簡單枚舉
- 跳過某些值
- 位元標記 (Bit Flags)
- 複雜表達式

### 不要使用iota的狀況

iota只適合

如果常數的value是一個已知的固定值, 例如 HTTP Status, 那就不適合iota
如果常數的value定義在外部的spec, 那也不適合iota, 因為iota



In [ ]:
package main

import "fmt"

// ======================================================================
// 基本 iota 用法
// ======================================================================

type Weekday int

const (
    Sunday Weekday = iota  // 0
    Monday                 // 1 (自動使用 iota)
    Tuesday                // 2
    Wednesday              // 3
    Thursday               // 4
    Friday                 // 5
    Saturday               // 6
)

// Weekday 的方法
func (w Weekday) String() string {
    days := []string{"Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"}
    if int(w) < len(days) {
        return days[w]
    }
    return "Unknown"
}

func (w Weekday) IsWeekend() bool {
    return w == Sunday || w == Saturday
}

// ======================================================================
// 從非零值開始
// ======================================================================

type Priority int

const (
    _ Priority = iota  // 跳過 0
    Low                // 1
    Medium             // 2
    High               // 3
    Critical           // 4
)

func (p Priority) String() string {
    switch p {
    case Low:
        return "Low"
    case Medium:
        return "Medium"
    case High:
        return "High"
    case Critical:
        return "Critical"
    default:
        return "Unknown"
    }
}

// ======================================================================
// iota 與表達式
// ======================================================================

type Size int

const (
    Small  Size = 1 << iota  // 1 << 0 = 1
    Medium                   // 1 << 1 = 2
    Large                    // 1 << 2 = 4
    XLarge                   // 1 << 3 = 8
)

// 檔案大小常數
type ByteSize int64

const (
    _           = iota                   // 跳過第一個值
    KB ByteSize = 1 << (10 * iota)     // 1 << (10*1) = 1024
    MB                                  // 1 << (10*2) = 1048576
    GB                                  // 1 << (10*3)
    TB                                  // 1 << (10*4)
    PB                                  // 1 << (10*5)
)

func (b ByteSize) String() string {
    switch {
    case b >= PB:
        return fmt.Sprintf("%.2f PB", float64(b)/float64(PB))
    case b >= TB:
        return fmt.Sprintf("%.2f TB", float64(b)/float64(TB))
    case b >= GB:
        return fmt.Sprintf("%.2f GB", float64(b)/float64(GB))
    case b >= MB:
        return fmt.Sprintf("%.2f MB", float64(b)/float64(MB))
    case b >= KB:
        return fmt.Sprintf("%.2f KB", float64(b)/float64(KB))
    default:
        return fmt.Sprintf("%d bytes", b)
    }
}

// ======================================================================
// 位元標記 (Bit Flags)
// ======================================================================

type Permission uint

const (
    Read Permission = 1 << iota  // 1 << 0 = 1   (二進制: 001)
    Write                        // 1 << 1 = 2   (二進制: 010)
    Execute                      // 1 << 2 = 4   (二進制: 100)
)

func (p Permission) String() string {
    var perms []string
    if p&Read != 0 {
        perms = append(perms, "Read")
    }
    if p&Write != 0 {
        perms = append(perms, "Write")
    }
    if p&Execute != 0 {
        perms = append(perms, "Execute")
    }
    if len(perms) == 0 {
        return "None"
    }
    return fmt.Sprintf("%v", perms)
}

func (p Permission) HasRead() bool    { return p&Read != 0 }
func (p Permission) HasWrite() bool   { return p&Write != 0 }
func (p Permission) HasExecute() bool { return p&Execute != 0 }

// ======================================================================
// 複雜的 iota 應用
// ======================================================================

type Status int

const (
    StatusIdle Status = iota * 10  // 0 * 10 = 0
    StatusRunning                  // 1 * 10 = 10
    StatusPaused                   // 2 * 10 = 20
    StatusStopped                  // 3 * 10 = 30
    StatusError                    // 4 * 10 = 40
)

func (s Status) String() string {
    switch s {
    case StatusIdle:
        return "Idle"
    case StatusRunning:
        return "Running"
    case StatusPaused:
        return "Paused"
    case StatusStopped:
        return "Stopped"
    case StatusError:
        return "Error"
    default:
        return fmt.Sprintf("Unknown(%d)", s)
    }
}

func main() {
    fmt.Println("=== 基本 iota 枚舉 ===")
    
    // 星期枚舉
    today := Tuesday
    fmt.Printf("Today is %s (value: %d)\n", today, today)  // 輸出: Today is Tuesday (value: 2)
    fmt.Printf("Is weekend: %t\n", today.IsWeekend())      // 輸出: Is weekend: false
    
    weekend := Saturday
    fmt.Printf("%s is weekend: %t\n", weekend, weekend.IsWeekend())  // 輸出: Saturday is weekend: true
    
    fmt.Println("\n=== 優先級枚舉 ===")
    
    priorities := []Priority{Low, Medium, High, Critical}
    for _, p := range priorities {
        fmt.Printf("Priority %s (value: %d)\n", p, p)
    }
    // 輸出: Priority Low (value: 1)
    //      Priority Medium (value: 2)
    //      Priority High (value: 3)
    //      Priority Critical (value: 4)
    
    fmt.Println("\n=== 檔案大小常數 ===")
    
    sizes := []ByteSize{512, 1500 * KB, 2.5 * GB, 1.2 * TB}
    for _, size := range sizes {
        fmt.Printf("%d bytes = %s\n", size, size)
    }
    // 輸出: 512 bytes = 512 bytes
    //      1536000 bytes = 1.46 MB
    //      2684354560 bytes = 2.50 GB
    //      1319413953331 bytes = 1.20 TB
    
    fmt.Println("\n=== 位元標記權限 ===")
    
    // 組合權限
    readWrite := Read | Write                    // 3 (二進制: 011)
    fullPermission := Read | Write | Execute     // 7 (二進制: 111)
    
    fmt.Printf("Read permission: %s (value: %d)\n", Read, Read)  // 輸出: Read permission: [Read] (value: 1)
    fmt.Printf("Read+Write: %s (value: %d)\n", readWrite, readWrite)  // 輸出: Read+Write: [Read Write] (value: 3)
    fmt.Printf("Full permission: %s (value: %d)\n", fullPermission, fullPermission)  // 輸出: Full permission: [Read Write Execute] (value: 7)
    
    // 檢查權限
    fmt.Printf("Full permission has read: %t\n", fullPermission.HasRead())      // 輸出: true
    fmt.Printf("Full permission has write: %t\n", fullPermission.HasWrite())    // 輸出: true
    fmt.Printf("Full permission has execute: %t\n", fullPermission.HasExecute()) // 輸出: true
    
    fmt.Println("\n=== 複雜表達式 ===")
    
    statuses := []Status{StatusIdle, StatusRunning, StatusPaused, StatusStopped, StatusError}
    for _, status := range statuses {
        fmt.Printf("Status: %s (value: %d)\n", status, status)
    }
    // 輸出: Status: Idle (value: 0)
    //      Status: Running (value: 10)
    //      Status: Paused (value: 20)
    //      Status: Stopped (value: 30)
    //      Status: Error (value: 40)
    
    fmt.Println("\n=== iota 的優勢 ===")
    fmt.Println("1. 自動遞增，避免手動維護數值")
    fmt.Println("2. 插入新值不需要重新編號")
    fmt.Println("3. 支援複雜表達式")
    fmt.Println("4. 編譯時計算，零執行時開銷")
    fmt.Println("5. 適合定義枚舉和常數組")
}